In [25]:
import datetime as dt
import pandas as pd
import matplotlib as mplt
import numpy as np
import os
import seaborn as sb
import sklearn as skl
import yfinance as yf

In [2]:
def add_region(market_index: str, market_index_df: pd.DataFrame) -> pd.DataFrame:
  region_idx= \
    { 'US & Canada' : ['^GSPC', '^DJI', '^IXIC', '^RUT','^GSPTSE'],
      'Latin America' : ['^BVSP', '^MXX', '^IPSA'],
      'East Asia' : ['^N225', '^HSI', '000001.SS', '399001.SZ', '^TWII', '^KS11'],
      'ASEAN & Oceania' : ['^STI', '^JKSE', '^KLSE','^AXJO',  '^NZ50'],
      'South & West Asia' : ['^BSESN', '^TA125.TA'],
      'Europe' : ['^FTSE', '^GDAXI', '^FCHI', '^STOXX50E','^N100', '^BFX']
    }
  reg = ""
  for region in region_idx:
    if (market_index in region_idx[region]):
      reg =  region
  market_index_df["Region"] = reg
  return market_index_df

In [3]:
def convert_month(market_index_df: pd.DataFrame) -> pd.DataFrame:
  months = {
        1.0:"Jan",
        2.0: "Feb",
        3.0: "Mar",
        4.0: "Apr",
        5.0: "May",
        6.0: "Jun",
        7.0: "Jul",
        8.0: "Aug",
        9.0: "Sep",
        10.0: "Oct",
        11.0: "Nov",
        12.0: "Dec",
    }
  market_index_df["Month"] = market_index_df["Month"].map(months)
  return market_index_df

In [4]:
def convert_day(market_index_df: pd.DataFrame) -> pd.DataFrame:
  days = {
    0.0: "Monday",
    1.0: "Tuesday",
    2.0: "Wednesday",
    3.0: "Thursday",
    4.0: "Friday",
    5.0: "Saturday",
    6.0: "Sunday"
}

  market_index_df["Day of Week"] = market_index_df["Day of Week"].map(days)
  return market_index_df

In [5]:
def add_date(
    market_index_df: pd.DataFrame,
) -> pd.DataFrame:  # Add month and day of week features to dataframe
    # https://stackoverflow.com/questions/26105804/extract-month-from-date-in-python
    for date, row in market_index_df.iterrows():
        dtobj = date.date()
        market_index_df.at[date, "Month"] = dtobj.month
        market_index_df.at[date, "Day of Week"] = dtobj.weekday()
    market_index_df = convert_day(convert_month(market_index_df))
    return market_index_df

In [6]:
def adjust_close(market_index_df: pd.DataFrame) -> pd.DataFrame: #Update price to have a universal "currency"
  price_change = []
  for open_price, close_price in zip(market_index_df["Open"], market_index_df["Close"]): #Go through all open and closing prices
    price_change.append(((close_price - open_price) / open_price) * 100)
  market_index_df["Change In Price (%)"] = price_change
  return market_index_df

In [7]:
def market_index_df(
    market_index: str,
) -> (
    pd.DataFrame
):  # Creates a datafeame for a specific market index with additional attributes from the years 2010 to 2023
    market_index_df = []
    ticker_data = yf.Ticker(market_index)
    ticker_df = ticker_data.history(period="1d", start="2010-1-1", end="2023-12-31")
    market_index_df.append(ticker_df)
    market_index_df = pd.concat(market_index_df, axis=0)
    market_index_df = market_index_df.drop(
        columns=["Stock Splits", "Dividends"]
    )  # Dropping stock splits, dividends due to Market indices not having said features
    market_index_df = add_region(market_index, market_index_df)
    market_index_df = adjust_close(market_index_df)
    market_index_df = add_date(market_index_df)
    market_index_df.index = market_index_df.index.date
    return market_index_df

In [8]:
world_indices_page = pd.read_html("https://finance.yahoo.com/world-indices/")
world_indices = world_indices_page[0]
market_indices = {} #Dictionatry of market indices dataframes
for index in world_indices.Symbol:
  market_index_df = market_index_df(index)
  market_indices[index] = market_index_df
  break
display(market_indices["^GSPC"])

,Open,High,Low,Close,Volume,Region,Change In Price (%),Month,Day of Week
2010-01-04,1116.560059,1133.869995,1116.560059,1132.989990,3991400000,US & Canada,1.471478,Jan,Monday
2010-01-05,1132.660034,1136.630005,1129.660034,1136.520020,2491020000,US & Canada,0.340789,Jan,Tuesday
2010-01-06,1135.709961,1139.189941,1133.949951,1137.140015,4972660000,US & Canada,0.125917,Jan,Wednesday
2010-01-07,1136.270020,1142.459961,1131.319946,1141.689941,5270680000,US & Canada,0.476992,Jan,Thursday
2010-01-08,1140.520020,1145.390015,1136.219971,1144.979980,4389590000,US & Canada,0.391046,Jan,Friday
...,...,...,...,...,...,...,...,...,...
2023-10-23,4210.399902,4255.839844,4189.220215,4217.040039,3776100000,US & Canada,0.157708,Oct,Monday
2023-10-24,4235.790039,4259.379883,4219.430176,4247.680176,3821820000,US & Canada,0.280706,Oct,Tuesday
2023-10-25,4232.419922,4232.419922,4181.419922,4186.770020,3869370000,US & Canada,-1.078577,Oct,Wednesday
2023-10-26,4175.990234,4183.600098,4127.899902,4137.229980,4277640000,US & Canada,-0.928169,Oct,Thursday


In [9]:
def get_impactful_events() -> (
    dict
):  # Function to find the most impactful events on Stock Market from 2020 to 2023
    high_impact_times = {}
    covid = ["2020-1-1", "2023-05-11"]
    russia_ukraine = ["2022-02-24", "2022-04-07"]
    third_infitada = ["2023-10-07", "2023-12-31"]
    high_impact_times["COVID"] = covid
    high_impact_times["Russian_Ukranian_War"] = russia_ukraine
    high_impact_times["Israeli_Hamas_War"] = third_infitada
    return high_impact_times

In [10]:
impactful_events = get_impactful_events()

In [11]:
def create_gscpi_df(
    gscpi_index_df: pd.DataFrame,
) -> (
    pd.DataFrame
):  # Cleaning GSCPI index dataframe, removing nulls and prior CSV/XLS  gscpi_index_df = gscpi_index_df[["Date","GSCPI"]]
    gscpi_index_df = gscpi_index_df[["Date","GSCPI"]]
    # Gather all data ranging from 2010 to now
    gscpi_index_df = gscpi_index_df.drop(index = range(4))
    gscpi_index_df = gscpi_index_df.reset_index(drop = True)
    updated_dates = []
    updated_index = []
    for date, index in zip(gscpi_index_df["Date"], gscpi_index_df["GSCPI"]): # Gather all data ranging from 2010 to now
        if (int(date[7:]) >= 2010):
            updated_dates.append(date) 
            updated_index.append(index)
    gscpi_index_df["Date"] = pd.Series(updated_dates)
    gscpi_index_df["GSCPI"] = pd.Series(updated_index)
    gscpi_index_df =  gscpi_index_df.dropna(axis = 0)
    # Convert month format (name) to number for consistency between dataframes
    months = {
        "Jan": 1,
        "Feb": 2,
        "Mar": 3,
        "Apr": 4,
        "May": 5,
        "Jun": 6,
        "Jul": 7,
        "Aug": 8,
        "Sep": 9,
        "Oct": 10,
        "Nov": 11,
        "Dec": 12,
    }
    updated_dates = []
    for date in gscpi_index_df["Date"]:
        day, month, year = date.split("-")
        month_num = months[month]
        date = f"{year}-{month_num:02d}-{day}"
        updated_dates.append(date) 
    gscpi_index_df["Date"] = updated_dates
    
    index = pd.to_datetime(gscpi_index_df["Date"], format="%Y-%m-%d")
    gscpi_index_df = gscpi_index_df.set_index("Date")
    return gscpi_index_df



In [12]:
path = "C:\\Users\\millen11\\Dropbox\\PC\\Documents\\academia\\rpi\\classes\\fall23\\introToML\\introToMLapps\\project\\dataset\\pre-cleaned"

In [13]:
gscpi_df = pd.read_csv(path + '/gscpi_data.csv')
gscpi_df = create_gscpi_df(gscpi_df)
display(gscpi_df)

,GSCPI
Date,
2010-01-31,-0.30
2010-02-28,-0.14
2010-03-31,0.39
2010-04-30,0.29
2010-05-31,0.41
...,...
2023-05-31,-1.57
2023-06-30,-1.11
2023-07-31,-0.86


In [36]:
def create_ecpi_df(
    ecpi_df: pd.DataFrame,
) -> (
    pd.DataFrame
):  # Cleaning ECPI index dataframe, removing nulls and prior CSV/XLS formatting
    ecpi_df = ecpi_df.drop(index = range(201, 205))
    parsed_columns = []
    for column in ecpi_df.columns:
        if column.isdigit() and int(column) >= 201001:
            parsed_columns.append(column)
        elif column.isalpha():
            parsed_columns.append(column)
    ecpi_df = ecpi_df[parsed_columns]
    
    formatted_columns = []
    for column in ecpi_df.columns:
        year = column[:4]
        month = column[4:6]
        if ((column[-1] == "4") or (column[-1] == "6") or column[-1] == "9"):
            date_string = f"{year}-{month}-30"
            formatted_columns.append(date_string)
        elif (column[-1] == "2"):
            date_string = f"{year}-{month}-28"
            formatted_columns.append(date_string)
        else:
            date_string = f"{year}-{month}-31"
            formatted_columns.append(date_string)
    ecpi_df.columns = formatted_columns
    ecpi_df = ecpi_df.rename(columns = {"Coun-tr-31": "Country"})
    ecpi_df = ecpi_df.drop(columns = ["Note--31"])
    
    ecpi_df = ecpi_df.melt(id_vars = ["Country"], var_name = "Date", value_name = "ECPI")
    ecpi_df = ecpi_df.set_index("Date")
    
    ecpi_df = ecpi_df.dropna(axis = 0)
    
    return ecpi_df

In [37]:
ecpi_df = pd.read_csv(path + "/ecpi_montly.csv")
ecpi_df = create_ecpi_df(ecpi_df)
display(ecpi_df)

,Country,ECPI
Date,,
2010-01-31,Aruba,101.0
2010-01-31,United Arab Emirates,102.1
2010-01-31,Armenia,90.0
2010-01-31,Austria,91.9
2010-01-31,Azerbaijan,100.0
...,...,...
2023-03-31,"Taiwan, China",102.9
2023-03-31,Uganda,132.0
2023-03-31,Ukraine,713.9


In [52]:
def create_ccpi_df(
    ccpi_df: pd.DataFrame,
) -> (
    pd.DataFrame
):  # Cleaning CCPI index dataframe, removing nulls and prior CSV/XLS formatting
    parsed_columns = []
    for column in ccpi_df.columns:
        if column.isdigit() and int(column) >= 201001:
            parsed_columns.append(column)
        elif column.isalpha():
            parsed_columns.append(column)
    ccpi_df = ccpi_df[parsed_columns]
    
    formatted_columns = []
    for column in ccpi_df.columns:
        year = column[:4]
        month = column[4:6]
        if ((column[-1] == "4") or (column[-1] == "6") or column[-1] == "9"):
            date_string = f"{year}-{month}-30"
            formatted_columns.append(date_string)
        elif (column[-1] == "2"):
            date_string = f"{year}-{month}-28"
            formatted_columns.append(date_string)
        else:
            date_string = f"{year}-{month}-31"
            formatted_columns.append(date_string)
            
    ccpi_df.columns = formatted_columns
    ccpi_df = ccpi_df.rename(columns = {"Coun-tr-31": "Country"})
    ccpi_df = ccpi_df.drop(columns = ["Note--31"])
    
    ccpi_df = ccpi_df.melt(id_vars = ["Country"], var_name = "Date", value_name = "CCPI")
    ccpi_df = ccpi_df.set_index("Date")
    
    ccpi_df = ccpi_df.dropna(axis = 0)
    
    return ccpi_df

In [53]:
filename = 'ccpi_monthly.csv'
file_path = os.path.join(path, filename)
ccpi_df = pd.read_csv(file_path)
ccpi_df = create_ccpi_df(ccpi_df)
display(ccpi_df)

,Country,CCPI
Date,,
2010-01-31,Albania,90.2
2010-01-31,Austria,89.0
2010-01-31,Belgium,91.3
2010-01-31,Bulgaria,96.2
2010-01-31,Belarus,96.1
...,...,...
2023-03-31,Uganda,126.7
2023-03-31,Uganda,126.1
2023-03-31,Ukraine,341.8


In [55]:
def create_epsi_df(
    epsi_df: pd.DataFrame,
) -> pd.DataFrame:  # Cleaning EPSI index dataframe, removing nulls and formatting
    epsi_df = epsi_df.drop(
        columns = [
            "VAR",
            "Variable",
            "YEA",
            "Unit Code",
            "Unit",
            "PowerCode Code",
            "PowerCode",
            "Reference Period Code",
            "Reference Period",
            "Flag Codes",
            "Flags",
            "COU"
        ]
    )
    epsi_df = epsi_df.rename(columns = {"Value": "EPSI"})
    return epsi_df

In [56]:
filename = 'epsi.csv'
file_path = os.path.join(path, filename)
epsi_df = pd.read_csv(file_path)
epsi_df = create_epsi_df(epsi_df)
display(epsi_df)

,Country,Year,EPSI
0,Greece,1990,1.0
1,Greece,1991,1.0
2,Greece,1992,1.0
3,Greece,1993,1.0
4,Greece,1994,1.0
...,...,...,...
17386,South Africa,2016,0.0
17387,South Africa,2017,0.0
17388,South Africa,2018,0.0
17389,South Africa,2019,0.0
